In [30]:
import java.security.MessageDigest
import kotlin.text.Charsets.UTF_8
import kotlin.reflect.KClass
import kotlin.reflect.KType
import kotlin.reflect.full.memberProperties


fun md5(str: String): ByteArray = MessageDigest.getInstance("MD5").digest(str.toByteArray(UTF_8))
fun ByteArray.toHex() = joinToString("") { "%02x".format(it) }

typealias MerkleClass = Map<String, Map<String, Map<String, Map<String, KType>>>>
    
fun <T:Any> hash(x: KClass<T>) : MerkleClass {

    val name = x.simpleName.toString()
    var current  = StringBuilder().append("CLASSNAME: $name\n")
    
    val members = x.memberProperties.associate { 
        m -> run {
            val member_sig = "\t${m.name}: ${m.returnType.toString()}\n"
            current.append(member_sig.toString())           
            Pair(m.name, mapOf(md5(member_sig).toHex() to m.returnType))
        }
    }    
    println(current)
    return mapOf(name to mapOf(md5(current.toString()).toHex() to members))
}

In [31]:
class FooClass(val name: String, var enabled: Boolean)
val ref1 = hash(FooClass::class)
ref1


CLASSNAME: FooClass
	enabled: kotlin.Boolean
	name: kotlin.String



{FooClass={ba4f81e0cef6d917b6ee5740f7577e47={enabled={3a908a0c26b268ab658b751f4449211d=kotlin.Boolean}, name={2e85ed0051328c3b0da9270978c1b3c6=kotlin.String}}}}

In [36]:
class FooClass(val name: String, var enabled: Boolean, var bing: String)
val ref2 = hash(FooClass::class)
ref2

CLASSNAME: FooClass
	bing: kotlin.String
	enabled: kotlin.Boolean
	name: kotlin.String



{FooClass={ab0688e95773cc5851cd722fac75a468={bing={4bb76c4d0ac721567fee46b8e150013f=kotlin.String}, enabled={3a908a0c26b268ab658b751f4449211d=kotlin.Boolean}, name={2e85ed0051328c3b0da9270978c1b3c6=kotlin.String}}}}

In [21]:
ref1 != ref2

true

In [89]:
fun merge(c1:MerkleClass, c2:MerkleClass) : MerkleClass {
    return (c1.asSequence() + c2.asSequence())
        .distinct()
        .groupBy( {it.key}, {it.value})
        .mapValues { (_, values)-> values.associateBy({it.keys.first()},{it.values.first()})}       
}

In [90]:
merge(ref1,ref2)

{FooClass={ba4f81e0cef6d917b6ee5740f7577e47={enabled={3a908a0c26b268ab658b751f4449211d=kotlin.Boolean}, name={2e85ed0051328c3b0da9270978c1b3c6=kotlin.String}}, ab0688e95773cc5851cd722fac75a468={bing={4bb76c4d0ac721567fee46b8e150013f=kotlin.String}, enabled={3a908a0c26b268ab658b751f4449211d=kotlin.Boolean}, name={2e85ed0051328c3b0da9270978c1b3c6=kotlin.String}}}}